In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [9]:
df = pd.read_csv("../Data/Raw/new_approach/Weekly_Covid_Data_Population_Normalized.csv")

In [10]:
print(df.columns)
print(df.shape)

Index(['week_no', 'iso_code', 'new_cases', 'new_deaths', 'new_vaccinations',
       'new_people_vaccinated', 'reproduction_rate', 'stringency_index',
       'excess_mortality', 'population_density', 'median_age', 'aged_65_older',
       'aged_70_older', 'cardiovasc_death_rate', 'diabetes_prevalence',
       'female_smokers', 'male_smokers', 'hospital_beds_per_thousand',
       'life_expectancy', 'human_development_index', 'population'],
      dtype='object')
(41184, 21)


In [33]:

df.drop(["iso_code", "week_no"], axis=1, inplace=True)
#df.drop(columns=df.columns[0:1], axis=1, inplace=True)
pop = df.iloc[0]["population"]
last_country = df.iloc[234*176 - 1]
print(last_country)
cnt = df["population"].value_counts()

KeyError: "['iso_code', 'week_no'] not found in axis"

In [12]:
# Here we are going to normilze the data.
from sklearn import preprocessing

x = df.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)

In [13]:
# now we want to join datas together
prediction_days_count = 2
features_list = []
X_list = []
Y_list = []
for i in range(234): # 234 is number of countries
    for j in range(176): # 176 is number of weeks we have
        row_list = df.loc[i*176 + j, :].values.flatten().tolist()
        features_list.append(row_list)
# print(len(features_list))
# print((len(features_list) - prediction_days_count) + 1)
for i in range((len(features_list) - prediction_days_count) + 1):
    fl = features_list[i]
    fl_len = len(features_list[i])
    this_population = features_list[i][fl_len - 1]
    for j in range(1, prediction_days_count):
        if this_population == features_list[i + j][fl_len - 1]:
            fl.extend(features_list[i + j])
        else:
            this_population = -1
            break
    if this_population != -1:
        X_list.append(fl)
        Y_list.append(features_list[(i + prediction_days_count) - 1][0])


In [14]:
print(len(X_list))
print(len(Y_list))
np.shape(X_list)

40950
40950


(40950, 38)

In [71]:
import math
def train_test_splitter(X, Y, prediction_days_count, batch_count, batch_length):
    xlen = len(X)
    xs_count_for_each_country = int(xlen / 234)
    X_train = []
    X_test = []
    Y_train = []
    Y_test = []
    steps = math.floor((xs_count_for_each_country - (batch_count * batch_length)) / (batch_count + 1)) 
    i = 0
    #print(steps)
    for i in range(234):
        j = 0
        for j in range((i * xs_count_for_each_country),(i * xs_count_for_each_country) + steps):
            X_train.append(X[j])
        #print(j)
            Y_train.append(Y[j])
        while(j + batch_length + steps < (i+1) * xs_count_for_each_country):
            for k in range(j + prediction_days_count, j + batch_length - prediction_days_count):
                X_test.append(X[k])
                Y_test.append(Y[k])
            j += batch_length
            for k in range(j, j + steps):
                X_train.append(X[k])
                Y_train.append(Y[k])
            j += steps + 1
            while j < (i+1) * xs_count_for_each_country:
                X_train.append(X[j])
                Y_train.append(Y[j])
                j += 1
        #print(j)
    ########
    # for i in range(steps):
    #     X_train.append(X[i])
    # while(i + batch_length + steps < xlen):
    #     for j in range(batch_length):
    #         X_test.append(X[j])
    #         Y_test.append(Y[j])
    #     for j in range(i, i + steps):
    #         X_train.append(X[j])
    #         Y_train.append(Y[j])
    #     i += batch_length + steps
    return X_train, X_test, Y_train, Y_test
        

In [72]:
X_train, X_test, Y_train, Y_test = train_test_splitter(X_list, Y_list, prediction_days_count, 3, 10)


In [73]:
print(np.shape(X_train), np.shape(X_test), np.shape(Y_train), np.shape(Y_test))

(38610, 38) (1404, 38) (38610,) (1404,)


## SVR

In [18]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.svm import SVR

In [79]:
svr_rbf = SVR(kernel="rbf", C=100, gamma=0.1, epsilon=0.1)
svr_lin = SVR(kernel="linear", C=100, gamma="auto")
svr_poly = SVR(kernel="poly", C=100, gamma="auto", degree=3, epsilon=0.1, coef0=1)


In [91]:
y_pred.shape

(1404,)

In [93]:
y_te.shape

(1404,)

In [95]:
svrs = [svr_rbf, svr_lin, svr_poly]
#kernel_label = ["RBF", "Linear", "Polynomial"]
#model_color = ["m", "c", "g"]

X_tr = np.array(X_train)
y_tr = np.array(Y_train)
X_te = np.array(X_test)
y_te = np.array(Y_test)

for svr in (svrs):
    y_pred = svr.fit(X_tr, y_tr).predict(X_te)
    svr_score = svr_rbf.score( X_te, y_te)
    print(f"score of {svr}:" , svr_score)

score of SVR(C=100, gamma=0.1): -367.6108265874125
score of SVR(C=100, gamma='auto', kernel='linear'): -367.6108265874125
score of SVR(C=100, coef0=1, gamma='auto', kernel='poly'): -367.6108265874125


## KNN

In [96]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=2)
neigh.fit(X_tr, y_tr)


KNeighborsRegressor(n_neighbors=2)

In [ ]:
neigh.predict()

In [ ]:
neigh.score()

## Polynomial

In [97]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(2)
poly.fit_transform(X_train)
poly = PolynomialFeatures(interaction_only=True)
poly.fit_transform(X_tr)

array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        9.79393181e-01, 2.82625062e-02, 2.88314502e-02],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        9.79393181e-01, 2.82625062e-02, 2.88314502e-02],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        9.79393181e-01, 2.82625062e-02, 2.88314502e-02],
       ...,
       [1.00000000e+00, 6.88380804e-06, 6.75988704e-05, ...,
        9.76379716e-01, 1.11798068e-02, 1.14414359e-02],
       [1.00000000e+00, 1.17279693e-05, 6.75988704e-05, ...,
        9.76379716e-01, 1.11798068e-02, 1.14414359e-02],
       [1.00000000e+00, 2.11613358e-05, 1.35197741e-04, ...,
        9.76379716e-01, 1.11798068e-02, 1.14414359e-02]])

## AdaBoost

In [99]:
from sklearn.ensemble import AdaBoostRegressor
#from sklearn.datasets import make_regression

regr = AdaBoostRegressor(random_state=0, n_estimators=100)
regr.fit(X_tr, y_tr)
y_pred = regr.predict(X_te)
regr.score(X_te, y_pred)

1.0

## Random Forest

In [100]:
from sklearn.ensemble import RandomForestRegressor
#from sklearn.datasets import make_regression

regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(X_tr, y_tr)
print(regr.predict(X_te))

[0.00176535 0.00176535 0.00176535 ... 0.00176535 0.00176535 0.00176535]


## Stacking

In [101]:
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor

estimators = [
    ('lr', RidgeCV()),
    ('svr', LinearSVR(dual="auto", random_state=42))
]
reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(n_estimators=10,
                                          random_state=42)
)

reg.fit(X_tr, y_tr).score(X_te, y_te)

0.9999990260268121

# TEST

In [ ]:
# ns_list, train_scores, validation_scores = learning_curve(
#                                                    estimator = SVR(**opt_svr_param), 
#                                                    X = X, y = Y, 
#                                                    train_sizes = ns_list, cv = 5,
#                                                    scoring = 'neg_mean_squared_error')


In [ ]:
# train_scores, valid_scores = validation_curve(KNeighborsRegressor(), X, Y, 
#                                               param_name="n_neighbors",
#                                               param_range=k_list , cv=20, 
#                                               scoring = 'neg_mean_squared_error',
#                                               verbose=1, n_jobs=-1
#                                              )
